# Объединение таблиц - join

Изучим разные виды объединений таблиц и научимся находить элементы в одной таблице, которым соответствуют значения в другой.

Данные: мобильное приложение

installs — содержит данные об установках приложения по дням.

- DeviceID — идентификатор устройства, на которое было установлено приложение;
- InstallationDate — дата установки приложения;
- InstallCost — цена установки приложения в рублях;
- Platform — платформа, на которой было установлено приложение (iOS/ Android);
- Source — источник установки приложения (магазин приложения/ рекламная система/ переход с сайта).

events — содержит данные о том, как активно пользователи просматривают товары в приложении по дням.

- DeviceID — идентификатор устройства, на котором используется приложение;
- AppPlatform — платформа, на которой используется приложение (iOS/ Android);
- EventDate — дата, за которую собрана статистика;
- events — количество просмотров всех товаров за этот день у этого DeviceID.

checks — содержит данные о покупках пользователей в приложении по дням

- UserID — идентификатор пользователя;
- Rub — суммарный чек пользователя на дату;
- BuyDate — дата, за которую собрана статистика.

devices – чтобы просмотры и установки можно было объединить с покупками
 
- DeviceID — идентификатор устройства;
- UserID — идентификатор пользователя.

Особенность приложения заключается в том, что для просмотра товаров не обязательно авторизовываться. До момента авторизации про пользователя известен только его DeviceID — идентификатор устройства. При этом для совершения покупки логин обязателен. На моменте авторизации пользователю присваивается UserID, и тогда мы уже знаем два его идентификатора: DeviceID (устройство) и UserID (логин). Так как на этапах установки приложения и просмотра каталога пользователь еще может быть не авторизован, там мы сохраняем только DeviceID. Но так как покупки нельзя совершить без авторизации, то покупки сохраняются только с UserID. Для того чтобы просмотры и установки можно было объединить с покупками, нам нужна таблица соответствия DeviceID к UserID, то есть таблица devices

1) У пользователя может быть два идентификатора – UserID и DeviceID. В таблице checks есть только UserID, в остальных – только DeviceID. Во вспомогательной таблице devices есть и UserID, и DeviceID. 

Давайте с помощью JOIN дополним таблицу events (left) данными о UserID пользователей из таблицы devices (right). Для некоторых DeviceID не будет пары UserID из таблицы devices – подумайте, какой вид JOIN подойдет, чтобы не потерять те строки, где DeviceID есть в events, но нет в devices.

Укажите UserID из первой строки результирующей таблицы, используя сортировку по убыванию по полю DeviceID.

    SELECT 
        l.DeviceID,
        r.UserID

    FROM events AS l 
    LEFT JOIN devices AS r 
        ON l.DeviceID = r.DeviceID

    ORDER BY DeviceID DESC  

    LIMIT 1



2) Давайте проверим, пользователи пришедшие из какого источника совершили наибольшее число покупок. В качестве ответа выберите название Source, юзеры которого совершили больше всего покупок.

    SELECT  
        Source,
        count(*) AS NumChecks

    FROM
    (
    SELECT 
        l.UserID,
        l.DeviceID,
        r.Source

    FROM 
        (
        SELECT 
            l.UserID,
            r.DeviceID
        FROM checks AS l 
        LEFT JOIN devices AS r 
            ON l.UserID = r.UserID
        ) AS l 

    LEFT JOIN installs AS r
        ON l.DeviceID = r.DeviceID


    )

    GROUP BY Source 

    ORDER BY NumChecks DESC 

    LIMIT 1


3) Теперь выясним, сколько всего уникальных юзеров совершили покупки в нашем приложении.
Объедините нужные таблицы, посчитайте число уникальных UserID для каждого источника (Source), и в качестве ответа укажите число пользователей, пришедших из Source_7.

    SELECT  
        Source,
        uniqExact(c.UserID) 
    FROM
        installs AS i
    INNER JOIN devices AS d
        ON d.DeviceID = i.DeviceId
    INNER JOIN checks AS c
    ON c.UserId = d.UserId
    GROUP BY Source 
    HAVING Source = 'Source_7'


4) Самое время посмотреть на общую выручку, а также минимальный, максимальный и средний чек. Рассчитайте нужные показатели и соотнесите полученные значения

    SELECT  
        i.Source,
        SUM(Rub) AS Revenue,
        MIN(Rub) AS Min_Checks,
        MAX(Rub) AS Max_Checks,
        AVG(Rub) AS Avg_Checks
    FROM 
        installs AS i
    INNER JOIN 
        devices AS d
        ON d.DeviceID = i.DeviceId
    INNER JOIN 
    checks AS c
        ON c.UserId = d.UserId
    GROUP BY Source 
    ORDER BY Source 



5) Выведите идентификаторы устройств пользователей, которые совершили как минимум одну покупку за последний месяц (октябрь 2019). Используйте сортировку по возрастанию DeviceID и укажите минимальное значение.

    SELECT  
        d.DeviceID,
        toStartOfMonth(toDate(c.BuyDate)) AS BuyMonth

    FROM 
        checks AS c
    INNER JOIN 
        devices AS d
    ON c.UserID = d.UserID

    WHERE  
        BuyMonth = '2019-10-01'

    ORDER BY d.DeviceID ASC  
    LIMIT 1


6) Проверим, сколько товаров (events) в среднем просматривают пользователи с разных платформ (Platform), и пришедшие из разных источников  (Source). Для этого объедините таблицы events и installs, и посчитайте, сколько просмотров в среднем приходится на каждую пару платформа-канал привлечения.

Отсортируйте полученную табличку по убыванию среднего числа просмотров. В качестве ответа укажите платформу и источник, пользователи которого в среднем просматривали товары бóльшее число раз.



    SELECT  
        l.Platform,
        r.Source,
        AVG(l.events) AS Avg_events

    FROM events AS l 
        JOIN installs AS r 
            ON l.DeviceID = r.DeviceID

    GROUP BY 
        l.Platform,
        r.Source 

    ORDER BY Avg_events DESC  

    LIMIT 1



7) Давайте посчитаем число уникальных DeviceID в инсталлах, для которых присутствуют просмотры в таблице events с разбивкой по платформам (поле Platform). Для этого можно отобрать все строки только из таблицы installs, для которых нашлось соответствие в таблице events. 

В качестве ответа введите число инсталлов, для которых нашлись просмотры, на платформе android.

    SELECT  
        r.Platform,
        uniqExact(l.DeviceID)

        FROM installs AS l 
        LEFT SEMI JOIN events AS r 
            ON l.DeviceID = r.DeviceID

    WHERE
        r.Platform = 'android'

    GROUP BY 
        r.Platform


8) Давайте теперь посчитаем конверсию из инсталла в просмотр с разбивкой по платформе инсталла – в данном случае это доля DeviceID, для которых есть просмотры, от всех DeviceID в инсталлах. 

Для этого нужно объединить таблицы installs и events так, чтобы получить все DeviceID инсталлов и соответствующие им DeviceID из events, посчитать число уникальных DeviceID инсталлов (1) и соответствующих DeviceID из events (2) и вычислить долю (2) от (1). В качестве ответа укажите значение конверсии из инсталла в просмотр на платформе ios. 


    SELECT  
        r.Platform,
        (uniqExact(r.DeviceID) / uniqExact(l.DeviceID)) AS Conversion 

        FROM installs AS l 
        LEFT JOIN events AS r 
            ON l.DeviceID = r.DeviceID

    WHERE
        Platform = 'iOS'

    GROUP BY 
        Platform



9) Представим, что в логирование DeviceID в событиях закралась ошибка - часть ID была записана в базу некорректно. Это привело к тому, что в таблице с событиями появились DeviceID, для которых нет инсталлов. Нам надо отобрать примеры DeviceID из таблицы event, которых нет в таблице installs, чтобы отправить их команде разработчиков на исправление. 

Выведите 10 уникальных DeviceID, которые присутствуют в таблице events, но отсутствуют в installs, отсортировав их в порядке убывания. 

    SELECT  
        DISTINCT DeviceID

    FROM events AS l 
        LEFT ANTI JOIN installs AS r 
            ON l.DeviceID = r.DeviceID

    ORDER BY  
        DeviceID DESC 

    LIMIT 10
